In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import math
import subprocess
import os
import scipy.optimize as opt
from numpy import exp
import timeit
from scipy.optimize import fsolve

plt.rcParams['figure.dpi']= 100
plt.rc("savefig", dpi=300)
palette_colors = ['#3288bd','#d53e4f', '#99d594', '#fc8d59' ]

pi = math.pi

plt.rcParams['text.usetex'] = True

ModuleNotFoundError: No module named 'numpy'

## Crack opening comparison



The effective crack opening is given as (Yoshioka et al. 2020):

For AT1
\begin{equation}
a^{\text{eff}} = a \left( 1 + \frac{\pi \ell}{4a (3h/8\ell + 1)} \right).
\label{eq:Eff_len_AT1}
\end{equation}
For AT2
\begin{align}
a^{\text{eff}}  =  a \left( 1 + \frac{\pi \ell/4}{a(h/2\ell + 1)} \right) .
\label{eq:Eff_len_AT2}
\end{align}

Approximation of a "jump" is from (eq. 24 of Chukwudozie et al. 2019):
\begin{equation} 
\int_{-\infty}^{+\infty} \phi(x,y) | \nabla v(y)| \,\mathrm{d} y \approx \phi(x,0^{+}) - \phi(x,0^{-}) .
\end{equation}


For the normal component of the displacement, $\phi(x,y) = \mathbf{u} \cdot \mathbf{n}$

\begin{equation} 
\int_{-\infty}^{+\infty} \mathbf{u} \cdot \mathbf{n} | \nabla v(y)| \,\mathrm{d} y = \int_{-\infty}^{+\infty} \mathbf{u} \cdot \frac{\nabla v(y)}{| \nabla v(y)|} | \nabla v(y)| \,\mathrm{d} y
\end{equation}

So,
\begin{equation} 
|| \mathbf{u} \cdot \mathbf{n}|| = \mathbf{u} \cdot \mathbf{n}(x,0^{+}) - \mathbf{u} \cdot \mathbf{n}(x,0^{-})  = \int_{-\infty}^{+\infty} \mathbf{u} \cdot \nabla v(y) \,\mathrm{d} y
\end{equation}

This computation is confirmed by Sneddon's solution for mode-I opening:

\begin{equation} 
  u_y(x,0) = \frac{2\left(1-\nu^2\right) pa}{E} \sqrt{1-\frac{x^2}{a^2}}
\end{equation}

For the shear displacement, $|| \mathbf{u} \cdot \mathbf{m}|| $, we again use:
\begin{equation} 
|| \mathbf{u} \cdot \mathbf{m}|| = \int_{-\infty}^{+\infty} \mathbf{u} \cdot \mathbf{m} | \nabla v(y)|  \,\mathrm{d} y
\end{equation}


<!-- <img src="schematic.png"> -->


For mode-II opening, Sneddon's solution is  

\begin{equation} 
  u_x(x,0) = \frac{2\left(1-\nu^2\right) \sigma_{xy} a}{E} \sqrt{1-\frac{x^2}{a^2}}
\end{equation}


## Phase-field implementation

With the shear opening of $|| \mathbf{u} \cdot \mathbf{m}||$ and the normal stress is: 
\begin{align}
\sigma_{nn} & = \begin{cases}
0 & \text{if } || \mathbf{u} \cdot \mathbf{n}|| > 0 \\
\boldsymbol{\sigma}:(\mathbf{n} \otimes \mathbf{n})& \text{otherwise},
\end{cases}
\end{align} 

Then the friction dissipatation is given as:

$$\int_\Gamma\mu \sigma_{nn} || \mathbf{u} \cdot \mathbf{m}||$$

where $\mu$ is the coefficient of friction.
Note that we are not considering the ''stick'' condition yet.

Using the following approximation of a scalar variable in $\Gamma$:

\begin{equation*}
    \int_{\Gamma} f \mathrm{d} S \approx \int_{\Omega} f \frac{(1-v)^n}{2 c_n \ell} \mathrm{d} \Omega,
\end{equation*}

The energy functional is given as:

$$ \int_{\Omega \setminus \Gamma} \frac{1}{2} {\sigma}: {\varepsilon} + \int_\Gamma \mu \sigma_{nn} || \mathbf{u} \cdot \mathbf{m}|| + \int_\Gamma G_c  \approx \int_{\Omega} v^2 \frac{1}{2} {\sigma}: {\varepsilon} + \int_\Omega \frac{(1-v)^n}{2 c_n \ell}  \mu \sigma_{nn} || \mathbf{u} \cdot \mathbf{m}||  +\int_\Omega \frac{G_c}{4c_n} \left(\frac{(1-v)^n}{\ell}+ \ell |\nabla v|^2\right)$$

where $c_n:= \int_0^1 (1-\eta)^{n/2} \mathrm{d} \eta$.




# Center Crack under Shear loading

## Explicit-Sneddon

### Explicit

In [ ]:
# fixed x and y at y=0 except for crack 

inputfile_u = open('/Users/mollaali/Opening/explicit/Shear/half/results/shear_half_disp.csv','r')
data_upper_explicit = [row.strip().split(',') for row in inputfile_u]
x_upper_explicit = []
y_upper_explicit = []
ux_upper_explicit =[]
uy_upper_explicit = []
for i in range(len(data_upper_explicit)):
    if i > 0:
        ux_upper_explicit.append(float(data_upper_explicit[i][3]))
        uy_upper_explicit.append(float(data_upper_explicit[i][4]))
        x_upper_explicit.append(float(data_upper_explicit[i][0]))
        y_upper_explicit.append(float(data_upper_explicit[i][1]))
zipped_upper_explicit = sorted(zip(x_upper_explicit,y_upper_explicit,ux_upper_explicit,uy_upper_explicit))


ux_explicit=[]
uy_explicit=[]
x_explicit=[]


for i in range(len(zipped_upper_explicit)):
    ux_explicit.append(zipped_upper_explicit[i][2])
    uy_explicit.append(zipped_upper_explicit[i][3])
    x_explicit.append(zipped_upper_explicit[i][0])


### Sneddon

In [ ]:
E=1.
nu=0.3
s=.008
x_Sneddon= np.linspace(-0.5*aa,0.5*aa,100)
ux_Sneddon=[]
for i in range(len(x_Sneddon)):
    ux_Sneddon.append(2*0.5*aa*(1-nu**2)*s/E*math.sqrt(1-((x_Sneddon[i])/(0.5*aa))**2))

### Plot

In [ ]:
plt.xlabel('$x$',fontsize =14)
plt.ylabel(r'$U_x$',fontsize=14)

plt.plot(np.array(x_Sneddon[:])/(0.5*aa), np.array(ux_Sneddon[:])/(2*0.5*aa*(1-nu**2)*s/E), 'k',label='Sneddon, Mode II')
plt.plot(np.array(x_explicit[:])/(0.5*aa), np.array(ux_explicit[:])/(2*0.5*aa*(1-nu**2)*s/E), 'y',label='Explicit-half, Mode II')

plt.grid(True)
legend = plt.legend()
plt.savefig('figures/shearSneddonExplicit.png')  

## Phase field-Sneddon

### Sneddon

In [ ]:
aa=.1*2 # for explicit-sneddon case
a = .1*2
h = 0.0025
ell = 2*h
a_eff = a*(1+pi*ell/(4.*a*(3*h/8./ell+1)))
a_eff/2


In [ ]:
E=1.
nu=0.0
s=.005

x_Sneddon_eff_shear= np.linspace(-0.5*a_eff,0.5*a_eff,100)
ux_Sneddon_eff_shear=[]
ux_max=2*0.5*a_eff*(1-nu**2)*s/E
for i in range(len(x_Sneddon_eff_shear)):
    ux_Sneddon_eff_shear.append(2*0.5*a_eff*(1-nu**2)*s/E*math.sqrt(1-((x_Sneddon_eff_shear[i])/(0.5*a_eff))**2))

### phase field

In [ ]:
inputfile_w_pf_Shear_half_Iso_h0p0025_l2h = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf/Ell_h_Const/half_0p0025_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0025_l2h = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0025_l2h]
width_pf_Shear_half_Iso_h0p0025_l2h = []
x_w_pf_Shear_half_Iso_h0p0025_l2h = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0025_l2h)):
        x_w_pf_Shear_half_Iso_h0p0025_l2h.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h[i][0]))
        width_pf_Shear_half_Iso_h0p0025_l2h.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h[i][1])) 
        

        
        inputfile_w_pf_Shear_half_Iso_h0p00125_l2h = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf/Ell_h_Const/half_0p00125_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p00125_l2h = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p00125_l2h]
width_pf_Shear_half_Iso_h0p00125_l2h = []
x_w_pf_Shear_half_Iso_h0p00125_l2h = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p00125_l2h)):
        x_w_pf_Shear_half_Iso_h0p00125_l2h.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h[i][0]))
        width_pf_Shear_half_Iso_h0p00125_l2h.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h[i][1]))   


        
inputfile_w_pf_Shear_half_Iso_h0p000625_l2h = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf/Ell_h_Const/half_0p000625_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p000625_l2h = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p000625_l2h]
width_pf_Shear_half_Iso_h0p000625_l2h = []
x_w_pf_Shear_half_Iso_h0p000625_l2h = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p000625_l2h)):
        x_w_pf_Shear_half_Iso_h0p000625_l2h.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h[i][0]))
        width_pf_Shear_half_Iso_h0p000625_l2h.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h[i][1])) 
        
        
inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf/Ell_h_Const/half_0p0003125_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0003125_l2h = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h]
width_pf_Shear_half_Iso_h0p0003125_l2h = []
x_w_pf_Shear_half_Iso_h0p0003125_l2h = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0003125_l2h)):
        x_w_pf_Shear_half_Iso_h0p0003125_l2h.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h[i][0]))
        width_pf_Shear_half_Iso_h0p0003125_l2h.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h[i][1])) 

### phase field

In [ ]:
   
plt.xlabel('$x/a_0$',fontsize =14)
plt.ylabel(r'$U_x/U_x^{max}$',fontsize=14)

plt.plot(np.array(x_Sneddon_eff_shear[:])/(0.5*a), np.array(ux_Sneddon_eff_shear[:])/ux_max, 'k',label='Sneddon')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0025_l2h[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p0025_l2h[:])/ux_max, 'g-.',label='Iso-half, $h=2.5$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p00125_l2h[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p00125_l2h[:])/ux_max, 'c-.',fillstyle='none',label='Iso-half, $h=1.25$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p000625_l2h[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p000625_l2h[:])/ux_max, 'b-.',fillstyle='none',label='Iso-half, $h=0.625$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0003125_l2h[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p0003125_l2h[:])/ux_max, 'r-.',fillstyle='none',label='Iso-half, $h=0.3125$ x $10^{-3}$')


   
plt.grid(True)
legend = plt.legend(loc='upper center', bbox_to_anchor=(1.25, 1.),
          fancybox=True, shadow=True, ncol=1)
plt.title('Initial Phase field: $h/2$ and $\ell=2h$')
plt.savefig('figures/shearSneddonPhaseField.png')  

In [ ]:
inputfile_w_pf_Shear_half_Iso_h0p0025_l2h_nu0 = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf_nu0/Ell_h_Const/half_0p0025_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0025_l2h_nu0 = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0025_l2h_nu0]
width_pf_Shear_half_Iso_h0p0025_l2h_nu0 = []
x_w_pf_Shear_half_Iso_h0p0025_l2h_nu0 = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0025_l2h_nu0)):
        x_w_pf_Shear_half_Iso_h0p0025_l2h_nu0.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h_nu0[i][0]))
        width_pf_Shear_half_Iso_h0p0025_l2h_nu0.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h_nu0[i][1])) 
        

        
        inputfile_w_pf_Shear_half_Iso_h0p00125_l2h_nu0 = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf_nu0/Ell_h_Const/half_0p00125_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p00125_l2h_nu0 = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p00125_l2h_nu0]
width_pf_Shear_half_Iso_h0p00125_l2h_nu0 = []
x_w_pf_Shear_half_Iso_h0p00125_l2h_nu0 = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p00125_l2h_nu0)):
        x_w_pf_Shear_half_Iso_h0p00125_l2h_nu0.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h_nu0[i][0]))
        width_pf_Shear_half_Iso_h0p00125_l2h_nu0.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h_nu0[i][1]))   


        
inputfile_w_pf_Shear_half_Iso_h0p000625_l2h_nu0 = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf_nu0/Ell_h_Const/half_0p000625_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p000625_l2h_nu0 = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p000625_l2h_nu0]
width_pf_Shear_half_Iso_h0p000625_l2h_nu0 = []
x_w_pf_Shear_half_Iso_h0p000625_l2h_nu0 = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p000625_l2h_nu0)):
        x_w_pf_Shear_half_Iso_h0p000625_l2h_nu0.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h_nu0[i][0]))
        width_pf_Shear_half_Iso_h0p000625_l2h_nu0.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h_nu0[i][1])) 
        
        
inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h_nu0 = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf_nu0/Ell_h_Const/half_0p0003125_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0003125_l2h_nu0 = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h_nu0]
width_pf_Shear_half_Iso_h0p0003125_l2h_nu0 = []
x_w_pf_Shear_half_Iso_h0p0003125_l2h_nu0 = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0003125_l2h_nu0)):
        x_w_pf_Shear_half_Iso_h0p0003125_l2h_nu0.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h_nu0[i][0]))
        width_pf_Shear_half_Iso_h0p0003125_l2h_nu0.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h_nu0[i][1])) 

In [ ]:

plt.xlabel('$x/a_0$',fontsize =14)
plt.ylabel(r'$U_x/U_x^{max}$',fontsize=14)

plt.plot(np.array(x_Sneddon_eff_shear[:])/(0.5*a), np.array(ux_Sneddon_eff_shear[:])/ux_max, 'k',label='Sneddon')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0025_l2h_nu0[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p0025_l2h_nu0[:])/ux_max, 'g-.',label='Iso-half, $h=2.5$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p00125_l2h_nu0[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p00125_l2h_nu0[:])/ux_max, 'c-.',fillstyle='none',label='Iso-half, $h=1.25$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p000625_l2h_nu0[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p000625_l2h_nu0[:])/ux_max, 'b-.',fillstyle='none',label='Iso-half, $h=0.625$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0003125_l2h_nu0[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p0003125_l2h_nu0[:])/ux_max, 'r-.',fillstyle='none',label='Iso-half, $h=0.3125$ x $10^{-3}$')



plt.grid(True)
legend = plt.legend(loc='upper center', bbox_to_anchor=(1.25, 1.),
       fancybox=True, shadow=True, ncol=1)
plt.title('Initial Phase field: $h/2, \quad \ell=2h, \quad nu=0 $')
plt.savefig('figures/shearSneddonPhaseField.png')  

In [ ]:
inputfile_w_pf_Shear_half_Iso_h0p0025_l2h_initPFline = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFline/Ell_h_Const/half_0p0025_linePF/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0025_l2h_initPFline = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0025_l2h_initPFline]
width_pf_Shear_half_Iso_h0p0025_l2h_initPFline = []
x_w_pf_Shear_half_Iso_h0p0025_l2h_initPFline = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0025_l2h_initPFline)):
        x_w_pf_Shear_half_Iso_h0p0025_l2h_initPFline.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h_initPFline[i][0]))
        width_pf_Shear_half_Iso_h0p0025_l2h_initPFline.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h_initPFline[i][1])) 
        

        
        inputfile_w_pf_Shear_half_Iso_h0p00125_l2h_initPFline = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFline/Ell_h_Const/half_0p00125_linePF/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p00125_l2h_initPFline = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p00125_l2h_initPFline]
width_pf_Shear_half_Iso_h0p00125_l2h_initPFline = []
x_w_pf_Shear_half_Iso_h0p00125_l2h_initPFline = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p00125_l2h_initPFline)):
        x_w_pf_Shear_half_Iso_h0p00125_l2h_initPFline.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h_initPFline[i][0]))
        width_pf_Shear_half_Iso_h0p00125_l2h_initPFline.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h_initPFline[i][1]))   


        
inputfile_w_pf_Shear_half_Iso_h0p000625_l2h_initPFline = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFline/Ell_h_Const/half_0p000625_linePF/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p000625_l2h_initPFline = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p000625_l2h_initPFline]
width_pf_Shear_half_Iso_h0p000625_l2h_initPFline = []
x_w_pf_Shear_half_Iso_h0p000625_l2h_initPFline = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p000625_l2h_initPFline)):
        x_w_pf_Shear_half_Iso_h0p000625_l2h_initPFline.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h_initPFline[i][0]))
        width_pf_Shear_half_Iso_h0p000625_l2h_initPFline.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h_initPFline[i][1])) 
        
        
inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h_initPFline = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFline/Ell_h_Const/half_0p0003125_linePF/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0003125_l2h_initPFline = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h_initPFline]
width_pf_Shear_half_Iso_h0p0003125_l2h_initPFline = []
x_w_pf_Shear_half_Iso_h0p0003125_l2h_initPFline = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0003125_l2h_initPFline)):
        x_w_pf_Shear_half_Iso_h0p0003125_l2h_initPFline.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h_initPFline[i][0]))
        width_pf_Shear_half_Iso_h0p0003125_l2h_initPFline.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h_initPFline[i][1])) 

In [ ]:
plt.xlabel('$x/a_0$',fontsize =14)
plt.ylabel(r'$U_x/U_x^{max}$',fontsize=14)

plt.plot(np.array(x_Sneddon_eff_shear[:])/(0.5*a), np.array(ux_Sneddon_eff_shear[:])/ux_max, 'k',label='Sneddon')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0025_l2h_initPFline[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p0025_l2h_initPFline[:])/ux_max, 'g-.',label='Iso-half, $h=2.5$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p00125_l2h_initPFline[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p00125_l2h_initPFline[:])/ux_max, 'c-.',fillstyle='none',label='Iso-half, $h=1.25$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p000625_l2h_initPFline[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p000625_l2h_initPFline[:])/ux_max, 'b-.',fillstyle='none',label='Iso-half, $h=0.625$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0003125_l2h_initPFline[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p0003125_l2h_initPFline[:])/ux_max, 'r-.',fillstyle='none',label='Iso-half, $h=0.3125$ x $10^{-3}$')



plt.grid(True)
legend = plt.legend(loc='upper center', bbox_to_anchor=(1.25, 1.),
       fancybox=True, shadow=True, ncol=1)
plt.title('Initial Phase field: \emph{line} and $\ell=2h$')
plt.savefig('figures/shearSneddonPhaseField.png')  

In [ ]:
inputfile_w_pf_Shear_half_Iso_h0p0025_l0p005 = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf/Ell_Const/half_0p0025_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0025_l0p005 = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0025_l0p005]
width_pf_Shear_half_Iso_h0p0025_l0p005 = []
x_w_pf_Shear_half_Iso_h0p0025_l0p005 = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0025_l0p005)):
        x_w_pf_Shear_half_Iso_h0p0025_l0p005.append(float(data_w_pf_Shear_half_Iso_h0p0025_l0p005[i][0]))
        width_pf_Shear_half_Iso_h0p0025_l0p005.append(float(data_w_pf_Shear_half_Iso_h0p0025_l0p005[i][1])) 
        

        
        inputfile_w_pf_Shear_half_Iso_h0p00125_l0p005 = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf/Ell_Const/half_0p00125_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p00125_l0p005 = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p00125_l0p005]
width_pf_Shear_half_Iso_h0p00125_l0p005 = []
x_w_pf_Shear_half_Iso_h0p00125_l0p005 = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p00125_l0p005)):
        x_w_pf_Shear_half_Iso_h0p00125_l0p005.append(float(data_w_pf_Shear_half_Iso_h0p00125_l0p005[i][0]))
        width_pf_Shear_half_Iso_h0p00125_l0p005.append(float(data_w_pf_Shear_half_Iso_h0p00125_l0p005[i][1]))   


        
inputfile_w_pf_Shear_half_Iso_h0p000625_l0p005 = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf/Ell_Const/half_0p000625_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p000625_l0p005 = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p000625_l0p005]
width_pf_Shear_half_Iso_h0p000625_l0p005 = []
x_w_pf_Shear_half_Iso_h0p000625_l0p005 = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p000625_l0p005)):
        x_w_pf_Shear_half_Iso_h0p000625_l0p005.append(float(data_w_pf_Shear_half_Iso_h0p000625_l0p005[i][0]))
        width_pf_Shear_half_Iso_h0p000625_l0p005.append(float(data_w_pf_Shear_half_Iso_h0p000625_l0p005[i][1])) 
        
        
inputfile_w_pf_Shear_half_Iso_h0p0003125_l0p005 = open('/Users/mollaali/Opening/PhaseField/shear/half_iniPFElmHalf/Ell_Const/half_0p0003125_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0003125_l0p005 = [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0003125_l0p005]
width_pf_Shear_half_Iso_h0p0003125_l0p005 = []
x_w_pf_Shear_half_Iso_h0p0003125_l0p005 = []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0003125_l0p005)):
        x_w_pf_Shear_half_Iso_h0p0003125_l0p005.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l0p005[i][0]))
        width_pf_Shear_half_Iso_h0p0003125_l0p005.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l0p005[i][1])) 

In [ ]:

plt.xlabel('$x/a_0$',fontsize =14)
plt.ylabel(r'$U_x/U_x^{max}$',fontsize=14)

plt.plot(np.array(x_Sneddon_eff_shear[:])/(0.5*a), np.array(ux_Sneddon_eff_shear[:])/ux_max, 'k',label='Sneddon')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0025_l0p005[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p0025_l0p005[:])/ux_max, 'g-.',label='Iso-half, $h=2.5$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p00125_l0p005[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p00125_l0p005[:])/ux_max, 'c-.',fillstyle='none',label='Iso-half, $h=1.25$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p000625_l0p005[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p000625_l0p005[:])/ux_max, 'b-.',fillstyle='none',label='Iso-half, $h=0.625$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0003125_l0p005[:])/(0.5*a), np.array(width_pf_Shear_half_Iso_h0p0003125_l0p005[:])/ux_max, 'r-.',fillstyle='none',label='Iso-half, $h=0.3125$ x $10^{-3}$')



plt.grid(True)
legend = plt.legend(loc='upper center', bbox_to_anchor=(1.25, 1.),
       fancybox=True, shadow=True, ncol=1)
plt.title('Initial Phase field: $h/2$ and $\ell=Const.=0.005$')
plt.savefig('figures/shearSneddonPhaseField.png')  

# Diagonal pulling

## explicit - half

In [ ]:
# fixed x and y at y=0 except for crack 
inputfile_u_half_quad= open('/Users/mollaali/Opening/explicit/DiagonalPulling/half_a_eff_0p2016_quad/results/shear_half_disp_aeff.csv','r')


data_upper_explicit_half_quad = [row.strip().split(',') for row in inputfile_u_half_quad]
x_upper_explicit_half_quad = []
y_upper_explicit_half_quad = []
ux_upper_explicit_half_quad =[]
uy_upper_explicit_half_quad = []

for i in range(len(data_upper_explicit_half_quad)):
    if i > 0:
        ux_upper_explicit_half_quad.append(float(data_upper_explicit_half_quad[i][0]))
        uy_upper_explicit_half_quad.append(float(data_upper_explicit_half_quad[i][1]))
        x_upper_explicit_half_quad.append(float(data_upper_explicit_half_quad[i][2]))
        y_upper_explicit_half_quad.append(float(data_upper_explicit_half_quad[i][3]))
zipped_upper_explicit_half_quad = sorted(zip(x_upper_explicit_half_quad,y_upper_explicit_half_quad,ux_upper_explicit_half_quad,uy_upper_explicit_half_quad))


ux_explicit_half_quad=[]
uy_explicit_half_quad=[]
x_explicit_half_quad=[]


for i in range(len(zipped_upper_explicit_half_quad)):
    ux_explicit_half_quad.append(zipped_upper_explicit_half_quad[i][2])
    uy_explicit_half_quad.append(zipped_upper_explicit_half_quad[i][3])
    x_explicit_half_quad.append(zipped_upper_explicit_half_quad[i][0])
    
####################################    
# fixed x and y at y=0 except for crack 
inputfile_u_half_0p000625_quad= open('/Users/mollaali/Opening/explicit/DiagonalPulling/half_a_eff_0p2008_quad/results/shear_half_disp_aeff_0p2008.csv','r')


data_upper_explicit_half_0p000625_quad = [row.strip().split(',') for row in inputfile_u_half_0p000625_quad]
x_upper_explicit_half_0p000625_quad = []
y_upper_explicit_half_0p000625_quad = []
ux_upper_explicit_half_0p000625_quad =[]
uy_upper_explicit_half_0p000625_quad = []

for i in range(len(data_upper_explicit_half_0p000625_quad)):
    if i > 0:
        ux_upper_explicit_half_0p000625_quad.append(float(data_upper_explicit_half_0p000625_quad[i][0]))
        uy_upper_explicit_half_0p000625_quad.append(float(data_upper_explicit_half_0p000625_quad[i][1]))
        x_upper_explicit_half_0p000625_quad.append(float(data_upper_explicit_half_0p000625_quad[i][2]))
        y_upper_explicit_half_0p000625_quad.append(float(data_upper_explicit_half_0p000625_quad[i][3]))
zipped_upper_explicit_half_0p000625_quad = sorted(zip(x_upper_explicit_half_0p000625_quad,y_upper_explicit_half_0p000625_quad,ux_upper_explicit_half_0p000625_quad,uy_upper_explicit_half_0p000625_quad))


ux_explicit_half_0p000625_quad=[]
uy_explicit_half_0p000625_quad=[]
x_explicit_half_0p000625_quad=[]


for i in range(len(zipped_upper_explicit_half_0p000625_quad)):
    ux_explicit_half_0p000625_quad.append(zipped_upper_explicit_half_0p000625_quad[i][2])
    uy_explicit_half_0p000625_quad.append(zipped_upper_explicit_half_0p000625_quad[i][3])
    x_explicit_half_0p000625_quad.append(zipped_upper_explicit_half_0p000625_quad[i][0])






In [ ]:
# fixed x and y at y=0 except for crack 
inputfile_u_half= open('/Users/mollaali/Opening/explicit/DiagonalPulling/half_a_eff/results/shear_half_disp_aeff.csv','r')


data_upper_explicit_half = [row.strip().split(',') for row in inputfile_u_half]
x_upper_explicit_half = []
y_upper_explicit_half = []
ux_upper_explicit_half =[]
uy_upper_explicit_half = []

for i in range(len(data_upper_explicit_half)):
    if i > 0:
        ux_upper_explicit_half.append(float(data_upper_explicit_half[i][0]))
        uy_upper_explicit_half.append(float(data_upper_explicit_half[i][1]))
        x_upper_explicit_half.append(float(data_upper_explicit_half[i][2]))
        y_upper_explicit_half.append(float(data_upper_explicit_half[i][3]))
zipped_upper_explicit_half = sorted(zip(x_upper_explicit_half,y_upper_explicit_half,ux_upper_explicit_half,uy_upper_explicit_half))


ux_explicit_half=[]
uy_explicit_half=[]
x_explicit_half=[]


for i in range(len(zipped_upper_explicit_half)):
    ux_explicit_half.append(zipped_upper_explicit_half[i][2])
    uy_explicit_half.append(zipped_upper_explicit_half[i][3])
    x_explicit_half.append(zipped_upper_explicit_half[i][0])
    
####################################    
# fixed x and y at y=0 except for crack 
inputfile_u_half_0p000625= open('/Users/mollaali/Opening/explicit/DiagonalPulling/half_a_eff_0p2008/results/shear_half_disp_aeff_0p2008.csv','r')


data_upper_explicit_half_0p000625 = [row.strip().split(',') for row in inputfile_u_half_0p000625]
x_upper_explicit_half_0p000625 = []
y_upper_explicit_half_0p000625 = []
ux_upper_explicit_half_0p000625 =[]
uy_upper_explicit_half_0p000625 = []

for i in range(len(data_upper_explicit_half_0p000625)):
    if i > 0:
        ux_upper_explicit_half_0p000625.append(float(data_upper_explicit_half_0p000625[i][3]))
        uy_upper_explicit_half_0p000625.append(float(data_upper_explicit_half_0p000625[i][4]))
        x_upper_explicit_half_0p000625.append(float(data_upper_explicit_half_0p000625[i][0]))
        y_upper_explicit_half_0p000625.append(float(data_upper_explicit_half_0p000625[i][1]))
zipped_upper_explicit_half_0p000625 = sorted(zip(x_upper_explicit_half_0p000625,y_upper_explicit_half_0p000625,ux_upper_explicit_half_0p000625,uy_upper_explicit_half_0p000625))


ux_explicit_half_0p000625=[]
uy_explicit_half_0p000625=[]
x_explicit_half_0p000625=[]


for i in range(len(zipped_upper_explicit_half_0p000625)):
    ux_explicit_half_0p000625.append(zipped_upper_explicit_half_0p000625[i][2])
    uy_explicit_half_0p000625.append(zipped_upper_explicit_half_0p000625[i][3])
    x_explicit_half_0p000625.append(zipped_upper_explicit_half_0p000625[i][0])







## Phase field - half

In [ ]:
# Mode II opening

inputfile_w_pf_Shear_half_Iso_h0p0025_l2h_Slidding= open('/Users/mollaali/Opening/PhaseField/DiagonalPulling/half_iniPFElmHalf/Ell_h_Const/half_0p0025_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0025_l2h_Slidding= [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0025_l2h_Slidding]
width_pf_Shear_half_Iso_h0p0025_l2h_Slidding= []
x_w_pf_Shear_half_Iso_h0p0025_l2h_Slidding= []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0025_l2h_Slidding)):
        x_w_pf_Shear_half_Iso_h0p0025_l2h_Slidding.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h_Slidding[i][0]))
        width_pf_Shear_half_Iso_h0p0025_l2h_Slidding.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h_Slidding[i][1])) 
        

# Mode I opening
inputfile_w_pf_Shear_half_Iso_h0p0025_l2h_Open= open('/Users/mollaali/Opening/PhaseField/DiagonalPulling/half_iniPFElmHalf/Ell_h_Const/half_0p0025_halfElem/results/Pressureopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0025_l2h_Open= [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0025_l2h_Open]
width_pf_Shear_half_Iso_h0p0025_l2h_Open= []
x_w_pf_Shear_half_Iso_h0p0025_l2h_Open= []

print(len(data_w_pf_Shear_half_Iso_h0p0025_l2h_Open))
for i in range(len(data_w_pf_Shear_half_Iso_h0p0025_l2h_Open)):
        x_w_pf_Shear_half_Iso_h0p0025_l2h_Open.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h_Open[i][0]))
        width_pf_Shear_half_Iso_h0p0025_l2h_Open.append(float(data_w_pf_Shear_half_Iso_h0p0025_l2h_Open[i][1]))         
        

        
        
        
        
        
        
        # Mode II opening

inputfile_w_pf_Shear_half_Iso_h0p00125_l2h_Slidding= open('/Users/mollaali/Opening/PhaseField/DiagonalPulling/half_iniPFElmHalf/Ell_h_Const/half_0p00125_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p00125_l2h_Slidding= [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p00125_l2h_Slidding]
width_pf_Shear_half_Iso_h0p00125_l2h_Slidding= []
x_w_pf_Shear_half_Iso_h0p00125_l2h_Slidding= []

for i in range(len(data_w_pf_Shear_half_Iso_h0p00125_l2h_Slidding)):
        x_w_pf_Shear_half_Iso_h0p00125_l2h_Slidding.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h_Slidding[i][0]))
        width_pf_Shear_half_Iso_h0p00125_l2h_Slidding.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h_Slidding[i][1])) 
        

# Mode I opening
inputfile_w_pf_Shear_half_Iso_h0p00125_l2h_Open= open('/Users/mollaali/Opening/PhaseField/DiagonalPulling/half_iniPFElmHalf/Ell_h_Const/half_0p00125_halfElem/results/Pressureopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p00125_l2h_Open= [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p00125_l2h_Open]
width_pf_Shear_half_Iso_h0p00125_l2h_Open= []
x_w_pf_Shear_half_Iso_h0p00125_l2h_Open= []

print(len(data_w_pf_Shear_half_Iso_h0p00125_l2h_Open))
for i in range(len(data_w_pf_Shear_half_Iso_h0p00125_l2h_Open)):
        x_w_pf_Shear_half_Iso_h0p00125_l2h_Open.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h_Open[i][0]))
        width_pf_Shear_half_Iso_h0p00125_l2h_Open.append(float(data_w_pf_Shear_half_Iso_h0p00125_l2h_Open[i][1]))         
        
  






        
        # Mode II opening

inputfile_w_pf_Shear_half_Iso_h0p000625_l2h_Slidding= open('/Users/mollaali/Opening/PhaseField/DiagonalPulling/half_iniPFElmHalf/Ell_h_Const/half_0p000625_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p000625_l2h_Slidding= [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p000625_l2h_Slidding]
width_pf_Shear_half_Iso_h0p000625_l2h_Slidding= []
x_w_pf_Shear_half_Iso_h0p000625_l2h_Slidding= []

for i in range(len(data_w_pf_Shear_half_Iso_h0p000625_l2h_Slidding)):
        x_w_pf_Shear_half_Iso_h0p000625_l2h_Slidding.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h_Slidding[i][0]))
        width_pf_Shear_half_Iso_h0p000625_l2h_Slidding.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h_Slidding[i][1])) 
        

# Mode I opening
inputfile_w_pf_Shear_half_Iso_h0p000625_l2h_Open= open('/Users/mollaali/Opening/PhaseField/DiagonalPulling/half_iniPFElmHalf/Ell_h_Const/half_0p000625_halfElem/results/Pressureopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p000625_l2h_Open= [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p000625_l2h_Open]
width_pf_Shear_half_Iso_h0p000625_l2h_Open= []
x_w_pf_Shear_half_Iso_h0p000625_l2h_Open= []

print(len(data_w_pf_Shear_half_Iso_h0p000625_l2h_Open))
for i in range(len(data_w_pf_Shear_half_Iso_h0p000625_l2h_Open)):
        x_w_pf_Shear_half_Iso_h0p000625_l2h_Open.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h_Open[i][0]))
        width_pf_Shear_half_Iso_h0p000625_l2h_Open.append(float(data_w_pf_Shear_half_Iso_h0p000625_l2h_Open[i][1]))         
        
  



        
# Mode II opening

inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h_Slidding= open('/Users/mollaali/Opening/PhaseField/DiagonalPulling/half_iniPFElmHalf/Ell_h_Const/half_0p0003125_halfElem/results/Shearopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0003125_l2h_Slidding= [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h_Slidding]
width_pf_Shear_half_Iso_h0p0003125_l2h_Slidding= []
x_w_pf_Shear_half_Iso_h0p0003125_l2h_Slidding= []

for i in range(len(data_w_pf_Shear_half_Iso_h0p0003125_l2h_Slidding)):
        x_w_pf_Shear_half_Iso_h0p0003125_l2h_Slidding.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h_Slidding[i][0]))
        width_pf_Shear_half_Iso_h0p0003125_l2h_Slidding.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h_Slidding[i][1])) 
        

# Mode I opening
inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h_Open= open('/Users/mollaali/Opening/PhaseField/DiagonalPulling/half_iniPFElmHalf/Ell_h_Const/half_0p0003125_halfElem/results/Pressureopening.csv','r+')
data_w_pf_Shear_half_Iso_h0p0003125_l2h_Open= [row.strip().split('\t') for row in inputfile_w_pf_Shear_half_Iso_h0p0003125_l2h_Open]
width_pf_Shear_half_Iso_h0p0003125_l2h_Open= []
x_w_pf_Shear_half_Iso_h0p0003125_l2h_Open= []

print(len(data_w_pf_Shear_half_Iso_h0p0003125_l2h_Open))
for i in range(len(data_w_pf_Shear_half_Iso_h0p0003125_l2h_Open)):
        x_w_pf_Shear_half_Iso_h0p0003125_l2h_Open.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h_Open[i][0]))
        width_pf_Shear_half_Iso_h0p0003125_l2h_Open.append(float(data_w_pf_Shear_half_Iso_h0p0003125_l2h_Open[i][1]))         
        
  

## Plot

In [ ]:
plt.xlabel('$x$',fontsize =14)
plt.ylabel(r'$U$',fontsize=14)

plt.plot(np.array(x_explicit_half_quad[:]), np.array(ux_explicit_half_quad[:]), '-.k',fillstyle='none',label='Mode II, Explicit-half,$h=2.5$ x $10^{-3}$, $\ell=2h$, $a_{eff}=0.2033$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0025_l2h_Slidding[:]), np.array(width_pf_Shear_half_Iso_h0p0025_l2h_Slidding[:]), 'g-.',label='Mode II, Iso-half, $h=2.5$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p00125_l2h_Slidding[:]), np.array(width_pf_Shear_half_Iso_h0p00125_l2h_Slidding[:]), 'y-.',label='Mode II, Iso-half, $h=1.25$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p000625_l2h_Slidding[:]), np.array(width_pf_Shear_half_Iso_h0p000625_l2h_Slidding[:]), 'b-.',label='Mode II, Iso-half, $h=0.625$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0003125_l2h_Slidding[:]), np.array(width_pf_Shear_half_Iso_h0p0003125_l2h_Slidding[:]), 'r-.',label='Mode II, Iso-half, $h=0.3125$ x $10^{-3}$')






#################################
plt.plot(np.array(x_explicit_half_quad[:]), np.array(uy_explicit_half_quad[:]), '-k',fillstyle='none',label='Mode I, Explicit-half, $h=2.5$ x $10^{-3}$, $\ell=2h$,  $a_{eff}=0.2033$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0025_l2h_Open[:]), np.array(width_pf_Shear_half_Iso_h0p0025_l2h_Open[:]), 'g-',label='Mode I, Iso-half, $h=2.5$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p00125_l2h_Open[:]), np.array(width_pf_Shear_half_Iso_h0p00125_l2h_Open[:]), 'y-',label='Mode I, Iso-half, $h=1.25$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p000625_l2h_Open[:]), np.array(width_pf_Shear_half_Iso_h0p000625_l2h_Open[:]), 'b-',label='Mode I, Iso-half, $h=0.625$ x $10^{-3}$')
plt.plot(np.array(x_w_pf_Shear_half_Iso_h0p0003125_l2h_Open[:]), np.array(width_pf_Shear_half_Iso_h0p0003125_l2h_Open[:]), 'r-',label='Mode I, Iso-half, $h=0.3125$ x $10^{-3}$')





   
plt.grid(True)
legend = plt.legend(loc='upper center', bbox_to_anchor=(1.25, 1.),
          fancybox=True, shadow=True, ncol=1)
plt.title('Initial Phase field: $h/2$ and $\ell=2h$')


plt.savefig('figures/DiagPullModeI_II.png') 

In [ ]:
plt.xlabel('$x$',fontsize =14)
plt.ylabel(r'$U$',fontsize=14)

plt.plot(np.array(x_explicit_half_quad[:]), np.array(ux_explicit_half_quad[:]), '-g',fillstyle='none',label='Mode I, Quad. Elem.,  $h=2.5$x$10^{-3}$, $a_0=a_{eff}=0.2033$')
plt.plot(np.array(x_explicit_half_0p000625_quad[:]), np.array(ux_explicit_half_0p000625_quad[:]), '--k',fillstyle='none',label='Mode II,  Quad. Elem.,  $h=0.625$x$10^{-3}$, $\ell=2h$, $a_0=a_{eff}=0.2008$')





#################################
plt.plot(np.array(x_explicit_half_quad[:]), np.array(uy_explicit_half_quad[:]), '-r',fillstyle='none',label='Mode I, Quad. Elem.,  $h=2.5$x$10^{-3}$, $a_0=a_{eff}=0.2033$')
plt.plot(np.array(x_explicit_half_0p000625_quad[:]), np.array(uy_explicit_half_0p000625_quad[:]), '--b',fillstyle='none',label='Mode I, Quad. Elem.,  $h=0.625$x$10^{-3}$, $a_0=a_{eff}=0.2008$')




plt.title('Explicit-half, $\ell=2h$')


   
plt.grid(True)
legend = plt.legend(loc='upper center', bbox_to_anchor=(1.25, 1.),
          fancybox=True, shadow=True, ncol=1)
plt.savefig('figures/DiagPullModeI_II.png') 

In [ ]:
plt.xlabel('$x$',fontsize =14)
plt.ylabel(r'$U$',fontsize=14)

plt.plot(np.array(x_explicit_half_quad[:]), np.array(ux_explicit_half_quad[:]), '-g',fillstyle='none',label='Mode II, Quad. Elem.')
plt.plot(np.array(x_explicit_half[:]), np.array(ux_explicit_half[:]), '-.r',fillstyle='none',label='Mode II,   Quad. Elem. only in center')


#################################
plt.plot(np.array(x_explicit_half_quad[:]), np.array(uy_explicit_half_quad[:]), '-y',fillstyle='none',label='Mode I, Quad. Elem.')
plt.plot(np.array(x_explicit_half[:]), np.array(uy_explicit_half[:]), '-.b',fillstyle='none',label='Mode I,  Quad. Elem. only in center')



plt.title('Explicit-half,   $h=2.5$x$10^{-3}$, $\ell=2h$, $a_0=a_{eff}=0.2016$')


   
plt.grid(True)
legend = plt.legend(loc='upper center', bbox_to_anchor=(1.25, 1.),
          fancybox=True, shadow=True, ncol=1)
plt.savefig('figures/DiagPullModeI_II.png') 

In [ ]:
plt.xlabel('$x$',fontsize =14)
plt.ylabel(r'$U$',fontsize=14)



plt.plot(np.array(x_explicit_half_0p000625_quad[:]), np.array(ux_explicit_half_0p000625_quad[:]), '--k',fillstyle='none',label='Mode II,  Quad. Elem.')
plt.plot(np.array(x_explicit_half_0p000625[:]), np.array(ux_explicit_half_0p000625[:]), '-.c',fillstyle='none',label='Mode II,   Quad. Elem. only in center')





#################################


plt.plot(np.array(x_explicit_half_0p000625_quad[:]), np.array(uy_explicit_half_0p000625_quad[:]), '--b',fillstyle='none',label='Mode I, Quad. Elem.')
plt.plot(np.array(x_explicit_half_0p000625[:]), np.array(uy_explicit_half_0p000625[:]), '-.y',fillstyle='none',label='Mode I,  Quad. Elem. only in center')




plt.title('Explicit-half,   $h=0.625$x$10^{-3}$, $\ell=2h$, $a_0=a_{eff}=0.2008$')



   
plt.grid(True)
legend = plt.legend(loc='upper center', bbox_to_anchor=(1.25, 1.),
          fancybox=True, shadow=True, ncol=1)
plt.savefig('figures/DiagPullModeI_II.png') 